In [14]:
# pip install keras==3.4.1
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 1.7 MB/s eta 0:00:00


In [22]:
import pickle
import keras
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

from google.colab import drive
import nltk
nltk.download('punkt')
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
class UnitStandardizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Dictionary for standardizing units and descriptors
        self.standard_units = {
            'pounds': 'lb', 'pound': 'lb', 'lbs': 'lb', 'lb': 'lb',
            'quarts': 'qt', 'quart': 'qt', 'qts': 'qt', 'qt': 'qt',
            'cups': 'cup', 'cup': 'cup',
            'pints': 'pint', 'pint': 'pint',
            'gallons': 'gal', 'gallon': 'gal', 'gal': 'gal',
            'ounces': 'oz', 'ounce': 'oz', 'ozs': 'oz', 'oz': 'oz',
            'dozens': 'dozen', 'dozen': 'dozen',
            'tablespoons': 'tbsp', 'tablespoon': 'tbsp', 'tbsps': 'tbsp', 'tbsp': 'tbsp',
            'teaspoons': 'tsp', 'teaspoon': 'tsp', 'tsps': 'tsp', 'tsp': 'tsp',
            'loaves': 'loaf', 'loaf': 'loaf',
            'brownies': 'brownie', 'brownie': 'brownie',
            'meatloaves': 'meatloaf', 'meatloaf': 'meatloaf',
            'chimichangas': 'chimichanga', 'chimichanga': 'chimichanga',
            'pizzas': 'pizza', 'pizza': 'pizza',
            'crepes': 'crepe', 'crepe': 'crepe',
            'truffles': 'truffle', 'truffle': 'truffle',
            'rolls': 'roll', 'roll': 'roll',
            'pancakes': 'pancake', 'pancake': 'pancake',
            'muffins': 'muffin', 'muffin': 'muffin',
            'cookies': 'cookie', 'cookie': 'cookie',
            'cupcakes': 'cupcake', 'cupcake': 'cupcake',
            'bars': 'bar', 'bar': 'bar',
            'waffles': 'waffle', 'waffle': 'waffle',
            'meatballs': 'meatball', 'meatball': 'meatball',
            'biscuits': 'biscuit', 'biscuit': 'biscuit',
            'appetizers': 'appetizer', 'appetizer': 'appetizer',
            'breadsticks': 'breadstick', 'breadstick': 'breadstick',
            'buns': 'bun', 'bun': 'bun',
            'enchiladas': 'enchilada', 'enchilada': 'enchilada',
            'desserts': 'dessert', 'dessert': 'dessert',
            'servings': 'serving', 'serving': 'serving',
            'strawberries': 'strawberry', 'strawberry': 'strawberry',
            'quiches': 'quiche', 'quiche': 'quiche',
            'omelettes': 'omelette', 'omelette': 'omelette',
            'sandwiches': 'sandwich', 'sandwich': 'sandwich',
            'pretzels': 'pretzel', 'pretzel': 'pretzel',
            'mushrooms': 'mushroom', 'mushroom': 'mushroom',
            'taquitos': 'taquito', 'taquito': 'taquito',
            'batches': 'batch', 'batch': 'batch',
            'patties': 'patty', 'patty': 'patty',
            'sausages': 'sausage', 'sausage': 'sausage',
            'tamales': 'tamale', 'tamale': 'tamale',
            'lambs': 'lamb', 'lamb': 'lamb',
            'cheeseburgers': 'cheeseburger', 'cheeseburger': 'cheeseburger',
            'doughnuts': 'doughnut', 'doughnut': 'doughnut',
            'hotdogs': 'hotdog', 'hotdog': 'hotdog',
            'samosas': 'samosa', 'samosa': 'samosa',
            'pies': 'pie', 'pie': 'pie',
            'turkeys': 'turkey', 'turkey': 'turkey',
            'chickens': 'chicken', 'chicken': 'chicken',
            'potatoes': 'potato', 'potato': 'potato',
            'peppers': 'pepper', 'pepper': 'pepper',
            'eggplants': 'eggplant', 'eggplant': 'eggplant',
            'fillets': 'fillet', 'fillet': 'fillet',
            'squares': 'square', 'square': 'square',
            'slabs': 'slab', 'slab': 'slab',
            'wraps': 'wrap', 'wrap': 'wrap',
            'burritos': 'burrito', 'burrito': 'burrito',
            'slices': 'slice', 'slice': 'slice',
        }
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.apply(self.standardize_units)

    def standardize_units(self, value):
        try:
            tokens = word_tokenize(value.lower())
            standardized_tokens = [self.standard_units.get(token, token) for token in tokens]
            standardized_value = ' '.join(standardized_tokens)
            standardized_value = re.sub(r'(\d+)\s+(\d+)/(\d+)', lambda m: str(int(m.group(1)) + float(m.group(2)) / float(m.group(3))), standardized_value)
            return standardized_value
        except:
            return 'None'

# Custom transformer for tokenizing
class CustomTokenizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.apply(word_tokenize)

# Custom transformer for encoding tokens
class TokenEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        flat_tokens = list(chain.from_iterable(X))
        self.label_encoder = LabelEncoder()
        self.encoded_tokens = self.label_encoder.fit_transform([str(token) for token in flat_tokens])
        self.token_dict = dict(zip(flat_tokens, self.encoded_tokens.ravel()))
        return self

    def transform(self, X, y=None):
        sequences = X.apply(lambda tokens: [self.token_dict[str(token)] for token in tokens])
        # max_len = max(sequences.apply(len))
        padded_sequences = pad_sequences(sequences, maxlen=2, padding='post')
        return padded_sequences

In [8]:
model = keras.models.load_model('/content/drive/MyDrive/recipes/recipe.keras')

with open('/content/drive/MyDrive/recipes/pipeline.pkl', 'rb') as f:
    pipeline = pickle.load(f)

with open('/content/drive/MyDrive/recipes/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [24]:
data = {
    "Name": ["Bourbon Chicken"],
    "Description": [""],
    "Facts": [""],
    "Directions": [""],
    "Ingredients": [""],
    "Nutrition": [""],
    "Category": ["Chicken Breast"],
    "Rating": [4.58],
    "URL": ["https://www.food.com/recipe/bourbon-chicken-45809"],
    "Fact_Ready In": [35],
    "Fact_Yields": [None],
    "Fact_Ingredients": [11],
    "Fact_Serves": [4],
    "Serving Size": [330],
    "Servings Per Recipe": [4],
    "Calories from Fat DV": [42.0],
    "Calories from Fat": [220000.0],
    "Total Fat DV": [37.0],
    "Total Fat": [24500.0],
    "Saturated Fat DV": [32.0],
    "Saturated Fat": [6500.0],
    "Cholesterol DV": [48.0],
    "Cholesterol": [145.3],
    "Sodium DV": [65.0],
    "Sodium": [1573.0],
    "Total Carbohydrate DV": [7.0],
    "Total Carbohydrate": [23400.0],
    "Dietary Fiber DV": [1.0],
    "Dietary Fiber": [300.0],
    "Sugars DV": [85.0],
    "Sugars": [21500.0],
    "Protein DV": [100.0],
    "Protein": [50100.0]
}

def generate_recipe(model, tokenizer, seed_text, num_features, max_sequence_len=250):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')
  generated_text = seed_text
  last_word = ""
  for _ in range(max_sequence_len):
    # Make a prediction
    predicted_probs = model.predict([token_list, token_list, num_features], verbose=0)

    # Get the word with the highest probability
    predicted_word_index = predicted_probs[:, -1, :].argmax(axis=-1)[0]

    # Convert the index to a word
    output_word = tokenizer.index_word.get(predicted_word_index, '')

    # Break if end token is reached
    if output_word == "endseq" or not output_word:
      break

    # Avoid repeating the same word consecutively
    if output_word != last_word:
      generated_text += ' ' + output_word
      last_word = output_word

    # Append the word to the seed text
    seed_text += ' ' + output_word

    # Update token_list for the next prediction
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

  # Post-processing to clean up repetitive phrases and make the text more coherent
  generated_text = ' '.join(dict.fromkeys(generated_text.split()))
  return generated_text

# Generate a new recipe
seed_text = "chocolate cake with butter and honey"
num_features = pipeline.transform(pd.DataFrame(data))
num_features = tf.convert_to_tensor(num_features)
print(generate_recipe(model, tokenizer, seed_text, num_features))

chocolate cake with butter and honey, heat until warm cream sugar in medium bowl. beat stir heat smooth a large mixture of butter and flour into proper boil. mix the mixture and store in cooler.
